# Hadi Babalou - 810199380

Artificial Intelligence - CA#01: *Search Algorithms* - Spring 2023 \
In this notebook, a food delivery problem is defined and solved using the following search algorithms:
1. Breadth First Search
2. Iterative Deepening Search
3. A* Search
4. Weighted A* Search

First, we need to define ```Graph``` class. This class is used to represent the problem map as a graph. For this purpose, first we need to define the ```Edge``` class. This class is used to represent the edges of the graph. Each edge has five instance variables:
-   ```id```: The identifier of the edge (an integer).
-   ```u``` and ```v```: The endpoints of the edge, represented as integers.
-   ```looseness```: An integer indicating the looseness of the edge. This is initially set to 0.
-   ```type```: An instance of the EdgeType enumeration, indicating whether the edge is ```REGULAR``` or ```LOOSE```. This is initially set to ```REGULAR```.

In [1]:
from enum import Enum

class EdgeType(Enum):
    REGULAR = 0
    LOOSE = 1

class Edge:
    id: int
    u: int
    v: int
    looseness: int
    type: EdgeType

    def __init__(self, id: int, u: int, v:int):
        self.id = id
        self.u = u
        self.v = v
        self.looseness = 0
        self.type = EdgeType.REGULAR

    def set_as_loose(self, looseness: int) -> None:
        self.looseness = looseness
        self.type = EdgeType.LOOSE
        

Then we define the ```Node``` class. This class is used to represent a node in a graph. It has three instance variables:
-   ```id```: The identifier of the node (an integer).
-   ```type```: An instance of the NodeType enumeration, indicating the type of the node (```REGULAR```, ```ZIRAJ```, ```PIZZA``` or ```STUDENT```). This is initially set to ```REGULAR```.
-   ```edges```: A list of edges that are adjacent to the node. This list is initially empty.

In [2]:
class NodeType(Enum):
    REGULAR = 0
    ZIRAJ = 1
    PIZZA = 2
    STUDENT = 3

class Node:
    id: int
    type: NodeType
    edges: list[Edge]

    def __init__(self, id: int):
        self.id = id
        self.edges = []
        self.type = NodeType.REGULAR

    def add_edge(self, edge: Edge) -> None:
        self.edges.append(edge)

    def set_type(self, type: NodeType) -> None:
        self.type = type


Finally, we define the ```Graph``` class. This class is used to represent the problem map as a graph. It has the following instance variables:
-   ```n```: The number of nodes in the graph (an integer).
-   ```m```: The number of edges in the graph (an integer).
-   ```nodes```: A list of Node objects in the graph.
-   ```edges```: A list of Edge objects in the graph.
-   ```loose_edges```: A set of Edge objects that are marked as loose.
-   ```ziraj```: An integer indicating the node ID of the Ziraj.
-   ```students```: A dictionary that maps a student ID to a node ID.
-   ```student_demands```: A dictionary that maps a student node ID to a pizza node ID, representing the demand of each student.
-   ```students_priority```: A dictionary indicating the priority of each student over another. The Pizza of the student with the key has a higher priority than the Pizza of the student with the value.

In [3]:
class Graph:
    n: int
    m: int
    nodes: list[Node]
    edges: list[Edge]
    loose_edges: set[Edge]
    ziraj: int
    students: dict[int, int]                    # student_id: node_id
    student_demands: dict[int, int]             # student_node: pizza_node
    students_priority: dict[int, list[int]]     # student1_node: student2_node

    def __init__(self, n: int, m: int):
        self.n = n
        self.m = m
        self.nodes = [Node(i) for i in range(0, n)]
        self.edges = []
        self.loose_edges = set()
        self.ziraj = -1
        self.pizzas = set()
        self.students = dict()
        self.student_demands = dict()
        self.students_priority = dict()

    def __str__(self) -> str:
        _str_ = ""
        _str_ += f"n: {self.n}, m: {self.m}"
        
        return _str_

    
    def add_edge(self, edge: Edge) -> None:
        self.edges.append(edge)
        self.nodes[edge.u].add_edge(edge)
        self.nodes[edge.v].add_edge(edge)

    def set_loose_edge(self, edge_id: int, looseness: int) -> None:
        edge = self.edges[edge_id]
        edge.set_as_loose(looseness)
        self.loose_edges.add(edge)

    def set_ziraj(self, node_id: int) -> None:
        self.ziraj = node_id
        self.nodes[node_id].set_type(NodeType.ZIRAJ)

    def add_demand(self, student_id: int, student_node:int, pizza_node: int) -> None:
        self.students[student_id] = student_node
        self.nodes[student_node].set_type(NodeType.STUDENT)
        self.nodes[pizza_node].set_type(NodeType.PIZZA)
        self.student_demands[student_node] = pizza_node

    def add_priority(self, student1_id: int, student2_id: int) -> None:
        if self.students[student1_id] not in self.students_priority:
            self.students_priority[self.students[student1_id]] = []
        self.students_priority[self.students[student1_id]].append(self.students[student2_id])


The State class is used to represent a state in the search space. It has the following instance variables:
-   ```agent_loc```: The location of the agent (an integer).
-   ```done_students```: A set of student locations whose demands have already been satisfied by the agent.
-   ```current_pickup```: The Node ID of the pizzaria that the agent is currently carrying a pizza from. This is initially set to None.
-   ```loose_status```: A dictionary that maps an ```Edge``` object to the remaining time during which the edge can be traversed. This dictionary represents the looseness status of the edges.
-   ```path_so_far```: A list of node IDs that the agent has visited so far.
-   ```time_on_node```: The time spent by the agent at the current node (an integer).

In [4]:
class State:
    agent_loc: int
    done_pizzas: set[int]
    done_students: set[int]
    current_pickup: int
    loose_status: dict[int, int]
    path_so_far: list[int]
    time_on_node: int

    def __init__(self, 
                agent_loc: int, 
                done_pizzas: set[int], 
                done_students: set[int], 
                current_pickup: int,
                loose_status: dict[int, int],
                path_so_far: list[int],
                time_on_node: int, 
                ):
        self.agent_loc = agent_loc
        self.done_pizzas = done_pizzas
        self.done_students = done_students
        self.current_pickup = current_pickup
        self.loose_status = loose_status
        self.path_so_far = path_so_far
        self.time_on_node = time_on_node
        
    def __eq__(self, other: object) -> bool:
        return isinstance(other, State) and (
                self.agent_loc == other.agent_loc and 
                self.done_pizzas == other.done_pizzas and 
                self.done_students == other.done_students and 
                self.current_pickup == other.current_pickup and 
                self.time_on_node == other.time_on_node # and 
                # self.loose_status == other.loose_status and
                # self.path_so_far == other.path_so_far
            )
    
    def __str__(self) -> str:
        _str_ = ""
        _str_ += f"agent loc: {self.agent_loc}, "
        _str_ += f"path so far: {self.path_so_far}, "
        _str_ += f"current pickup: {self.current_pickup}, "
        _str_ += f"time on node: {self.time_on_node}, "
        _str_ += f"done pizzas: {self.done_pizzas}, "
        _str_ += f"done students: {self.done_students}, "
        return _str_

    def __hash__(self) -> int:
        _str_ = ""
        _str_ += f"{self.agent_loc}, "
        _str_ += f"{self.done_pizzas}, "
        _str_ += f"{self.done_students}, "
        _str_ += f"{self.current_pickup}, "
        _str_ += f"{self.time_on_node}, "
        return hash(_str_)
    
    def __lt__(self, other: object) -> bool:
        return isinstance(other, State) and (
                len(self.path_so_far) < len(other.path_so_far) or (
                len(self.path_so_far) == len(other.path_so_far) and self.heuristic() > other.heuristic())
        )

    def heuristic(self) -> float:
        pickup = 0 if (self.current_pickup == -1 or self.current_pickup == 0) else 1
        return len(self.done_students) + len(self.done_pizzas) + pickup

    def relax(self) -> None:
        for edge, remaining_time in self.loose_status.items():
            if remaining_time > 0:
                self.loose_status[edge] = remaining_time - 1


The ```Search``` class contains an instance of the ```Graph``` class and some methods that are used to solve the problem. 
-   ```is_goal``` method is used to check whether a given state is a goal state or not.
-   ```init_state``` method is used to initialize the initial state of the problem.
-   ```can_pickup``` method is used to check whether the agent can pickup a pizza from a given pizzaria or not (based on students priority).
-   ```next_states``` method is used to generate the next states from a given state. This method is used by the search algorithms to expand the search space. 

In [5]:
from copy import deepcopy

class Search:
    graph: Graph

    def __init__(self, graph: Graph):
        self.graph = graph

    def is_goal(self, state: State) -> bool:
        return (len(state.done_students) == len(self.graph.students))

    def init_state(self) -> State:
        _loose_status_ = {}
        for edge in self.graph.loose_edges:
            _loose_status_[edge.id] = 0

        return State(
            agent_loc=self.graph.ziraj,
            done_pizzas=set(),
            done_students=set(),
            current_pickup=-1,
            loose_status=_loose_status_,
            path_so_far=[self.graph.ziraj],
            time_on_node=0,
        )
    
    def can_pickup(self, state: State, pizza: int) -> bool:
        # check if we can pickup pizza based on state's students_priority
        demanding_student = -1
        for key, value in self.graph.student_demands.items():
            if value == pizza:
                demanding_student = key
        if demanding_student == -1:
            return False
        
        for high_priority, low_priorities in self.graph.students_priority.items():
            for low_priority in low_priorities:
                if demanding_student == low_priority:
                    if high_priority not in state.done_students:
                        return False

        return True

    def next_states(self, cur_state: State) -> list[State]:
        next_states = []
        current_node = self.graph.nodes[cur_state.agent_loc]

        # go to adjacent nodes
        for edge in current_node.edges:
            new_state = deepcopy(cur_state)
            next_node = deepcopy(current_node)

            if edge.v == current_node.id:
                next_node = self.graph.nodes[edge.u]
            elif edge.u == current_node.id:
                next_node = self.graph.nodes[edge.v]

            if edge.type == EdgeType.LOOSE:                    
                if cur_state.loose_status[edge.id] > 0:
                    # stay on current node
                    new_state = deepcopy(cur_state)
                    new_state.path_so_far.append(current_node.id)
                    new_state.time_on_node += 1
                    new_state.relax()
                    next_states.append(new_state)
                    continue
                else:
                    new_state.loose_status[edge.id] = edge.looseness + 1

            new_state.agent_loc = next_node.id
            new_state.path_so_far.append(next_node.id)
            new_state.time_on_node = 0

            if next_node.type == NodeType.STUDENT:
                if next_node.id not in new_state.done_students:
                    if new_state.current_pickup == self.graph.student_demands[next_node.id]:
                        new_state.done_students.add(next_node.id)
                        new_state.done_pizzas.add(new_state.current_pickup)
                        new_state.current_pickup = -1
            
            elif next_node.type == NodeType.PIZZA:
                if next_node.id not in new_state.done_pizzas:
                    if (self.can_pickup(new_state, next_node.id)):
                        # i can pickup pizza here or leave it. both of this states should be added to next_states
                        # this handles picking up pizza. leaving it will be handled outside of this scope
                        new_state2 = deepcopy(new_state)
                        new_state2.current_pickup = next_node.id
                        new_state2.relax()
                        next_states.append(new_state2)

            new_state.relax()
            next_states.append(new_state)
            
        return next_states
    

## Breadth First Search

Now we define the ```BFS``` function. This function is used to solve the problem using the Breadth First Search algorithm. It takes an instance of the ```Search``` class as input and returns a list of node IDs that the agent should visit to deliver all pizzas. If there is no solution, it returns ```None```. 

The function initializes the initial state and adds it to a frontier list, which initially contains only the initial state. It also initializes an explored set to keep track of states that have been already explored.

The function then enters into a loop which continues until the frontier list is non-empty. In each iteration of the loop, the function removes the first state from the frontier list and adds it to the explored set, indicating that it has been visited. The function then generates all the possible next states that can be reached from the current state using the ```next_states``` method of the Search object, and adds only those states which have not been already explored or added to the frontier list.

If the current state is a goal state, the function returns the path from the initial state to the goal state. If no goal state is found, the function returns an empty list.

In [6]:
def BFS(problem: Search) -> tuple[list[int], int]:

    init_state = problem.init_state()
    if problem.is_goal(init_state):
        return init_state.path_so_far, 0

    explored = set()
    frontier = [init_state]
    states_expanded = 0

    while frontier:
        state = frontier.pop(0)
        explored.add(state)
        states_expanded += 1

        if problem.is_goal(state):
            return state.path_so_far, states_expanded

        for next_state in problem.next_states(state):
            if next_state not in explored and next_state not in frontier:
                if problem.is_goal(next_state):
                    return next_state.path_so_far, states_expanded
                frontier.append(next_state)

    return [], states_expanded


## Iterative Deepening Search

The following function is used to solve the problem using the Iterative Deepening Search algorithm. It takes an instance of the ```Search``` class as input and returns a list of node IDs that the agent should visit to deliver all pizzas. If there is no solution, it returns ```None```. 

We Use ```Depth Limited Search``` to implement ```Iterative Deepening Search```. The ```Depth Limited Search``` function takes an instance of the ```Search``` class and a depth limit as input and returns a list of node IDs that the agent should visit to deliver all pizzas. If there is no solution, it returns ```None```. 

In each iteration of the ```IDS``` loop, the function calls the ```DLS``` function with the current depth limit. If the ```DLS``` function returns a non-empty list, it means that a solution has been found and the function returns the solution. Otherwise, it increases the depth limit by 1 and continues the loop.

In [7]:
def DLS(problem: Search, limit:int) -> tuple[list[int], int]:

    init_state = problem.init_state()
    if problem.is_goal(init_state):
        return init_state.path_so_far, 0

    explored: dict[State, int] = {}
    depth = 0
    frontier: list[(State, int)] = [(init_state, depth)]
    states_expanded = 0

    while frontier:
        # pop from the end of the list
        state, depth = frontier.pop()
        explored[state] = depth
        states_expanded += 1

        if problem.is_goal(state):
            return state.path_so_far, states_expanded

        if depth >= limit:
            continue

        for next_state in problem.next_states(state):
            if (next_state, depth + 1) not in frontier:
                if next_state not in explored or explored[next_state] > depth + 1:
                    explored[next_state] = depth
                    if problem.is_goal(next_state):
                        return next_state.path_so_far, states_expanded
                    frontier.append((next_state, depth + 1))

    return [], states_expanded

def IDS(problem: Search) -> tuple[list[int], int]:
    limit = 0
    states_expanded_sum = 0
    while True:
        print("::: limit=", limit, ",", end=" ")
        path, states_expanded = DLS(problem, limit)
        states_expanded_sum += states_expanded
        if path:
            print(":::")
            return path, states_expanded
        limit += 1


## (Weighted) A* Search

To solve the problem using the A* Search algorithm, we need to define a heuristic function. The heuristic function takes a state as input and returns an estimate of the cost of the cheapest path from the state to a goal state. In this problem, we define the heuristic function as follows:

In [8]:
def heuristic(state: State, problem: Search) -> float:
    pickup = 0 if (state.current_pickup == -1 or state.current_pickup == 0) else 1
    return (len(problem.graph.students) - len(state.done_students)) + (len(problem.graph.student_demands) - len(state.done_pizzas) - pickup)


The function first checks if the current pickup is valid or not. If it is not, the value of the pickup is set as 0, and if it is valid, the value is set as 1.

Then, the function calculates the heuristic value used to guide the search process to the solution. The function first calculates the difference between the length of all the students in the graph and the length of the done students in the current state, which is the number of students yet to receive their pizza. This value gives an estimate of the remaining work to be done.

Next, the function calculates the difference between the length of all the student demands in the graph and the length of the done pizzas in the current state, which is the number of pizzas yet to be delivered. This value also gives an estimate of the remaining work to be done.

Finally, the function subtracts the pickup value calculated earlier from the above two values and returns the resulting float value as the heuristic estimate for the remaining effort to reach the goal state.

The output of the heuristic function is definitely an underestimate of the remaining effort to reach the goal state. 

No matter what action we do, our heuristic function value will change by at most 1. This is because the heuristic function value is calculated based on the number of students and pizzas that have not been delivered yet. If we deliver a pizza to a student, the number of students and pizzas that have not been delivered will decrease by 1 by the pickup value will increase by 1. Also our cost for each action is 1, so by the definition of consistency: 
$$ Cost(A\ to\ B) \geq h(A) - h(B) $$
And thus, our heuristic function is consistent and therefore admissible.


The ```frontier_insert``` function is used to insert a state into the frontier list. It takes a state and a list of states as input and updates the list of states. 

This function inserts the state into the list in a way that the list remains sorted based on the sum of the cost of the path from the initial state to the state and the heuristic value of the state.

In [9]:
def frontier_insert(frontier: list[State], state: State, problem: Search, alpha: float):
    if len(frontier) == 0:
        frontier.append(state)
        return

    state_est = alpha * heuristic(state, problem) + len(state.path_so_far)
    for i in range(len(frontier)):
        curr_est = alpha * heuristic(frontier[i], problem) + len(frontier[i].path_so_far)
        if curr_est < state_est:
            frontier.insert(i, state)
            return

    frontier.append(state)


finally, the ```A*``` function is used to solve the problem using the A* Search algorithm. It takes an instance of the ```Search``` class as input and returns a list of node IDs that the agent should visit to deliver all pizzas. If there is no solution, it returns ```None```.

The function initializes the initial state and adds it to a frontier list, which initially contains only the initial state. It also initializes an explored set to keep track of states that have been already explored.

The function then enters into a loop which continues until the frontier list is non-empty. In each iteration of the loop, the function removes the first state from the frontier list and adds it to the explored set, indicating that it has been visited. The function then generates all the possible next states that can be reached from the current state using the ```next_states``` method of the Search object, and adds only those states which have not been already explored or added to the frontier list.

In [10]:
def ASTAR(problem: Search, alpha: float=1) -> tuple[list[int], int]:
    init_state = problem.init_state()
    if problem.is_goal(init_state):
        return init_state.path_so_far, 0

    explored = set()
    frontier = [init_state]
    states_expanded = 0

    while frontier:
        state = frontier.pop()
        explored.add(state)
        states_expanded += 1

        if problem.is_goal(state):
            return state.path_so_far, states_expanded

        for next_state in problem.next_states(state):
            if next_state not in explored and next_state not in frontier:
                frontier_insert(frontier, next_state, problem, alpha)

    return [], states_expanded
    

## Results

The ```get_input``` function is used to read the input from the input file and create an instance of the ```Search``` class. It takes the name of the input file as input and returns an instance of the ```Search``` class.

In [11]:
def get_input(testcase_path: str) -> Search:
    file = open(testcase_path, "r")
    
    n, m = map(int, file.readline().split())
    graph = Graph(n, m)

    for i in range(m):
        u, v = map(int, file.readline().split())
        edge = Edge(i, u-1, v-1)
        graph.add_edge(edge)

    h = int(file.readline())
    for _ in range(h):
        i, xi = map(int, file.readline().split())
        graph.set_loose_edge(i-1, xi)

    v = int(file.readline())
    graph.set_ziraj(v-1)

    s = int(file.readline())
    for student_id in range(1, s+1):
        p, q = map(int, file.readline().split())
        graph.add_demand(student_id, p-1, q-1)
        
    t = int(file.readline())
    for i in range(1, t+1):
        a, b = map(int, file.readline().split())
        graph.add_priority(a, b)

    file.close()
    return Search(graph)


The ```run``` function will run a search algorithm and prints some information such as the found solution, the number of nodes expanded, and the time taken to find the solution.

In [12]:
import time
from typing import Callable

TESTCASES_PATH = "testcase/"
TESTS_REPEAT = 1
TESTS_START = 0
TESTS_COUNT = 3

def run(algorithm: Callable[[Search], tuple[list[int], int]]) -> None:
    for i in range(TESTS_START, TESTS_START + TESTS_COUNT):
        testcase_path = TESTCASES_PATH + str(i+1) + ".txt"
        search = get_input(testcase_path)
        duration = 0
        path = []
        nodes_expanded = 0
        for j in range(TESTS_REPEAT):
            start = time.time()
            path, nodes_expanded = algorithm(search)
            end = time.time()
            duration += end - start
        duration /= TESTS_REPEAT

        print("testcase", i+1)
        if len(path) == 0:
            print("no path found")
        
        print("path cost:", len(path))
        for j in range (len(path)):
            if (j == len(path) - 1):
                print(path[j] + 1)
                break
            print(path[j] + 1,"->", end=" ")
    
        print("nodes expanded:", nodes_expanded)
        print("duration:", duration)
        if i != (TESTS_START + TESTS_COUNT - 1):
            print("---------------------------------")
        

Now we can run the search algorithms on the input files and see the results.

In [13]:
from functools import partial

print("Testing BFS algorithm:")
print()
run(BFS)
print()
print("================================================================================")
print()


Testing BFS algorithm:

testcase 1
path cost: 11
6 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 1 -> 7 -> 9 -> 2
nodes expanded: 229
duration: 0.07977700233459473
---------------------------------
testcase 2
path cost: 21
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 12 -> 6 -> 14 -> 1 -> 14 -> 12 -> 3 -> 8 -> 6 -> 4 -> 10 -> 11 -> 8 -> 3 -> 13
nodes expanded: 933
duration: 0.4219381809234619
---------------------------------
testcase 3
path cost: 32
15 -> 14 -> 8 -> 20 -> 13 -> 8 -> 18 -> 10 -> 8 -> 9 -> 16 -> 5 -> 15 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 13 -> 8 -> 17 -> 8 -> 18 -> 2 -> 18 -> 4 -> 14 -> 16
nodes expanded: 21556
duration: 16.32559299468994




In [14]:
print("Testing ASTAR algorithm with alpha=1")
print()
run(partial(ASTAR, alpha=1))
print()
print("================================================================================")
print()


Testing ASTAR algorithm with alpha=1

testcase 1
path cost: 11
6 -> 4 -> 10 -> 9 -> 7 -> 9 -> 2 -> 9 -> 1 -> 3 -> 8
nodes expanded: 104
duration: 0.04292726516723633
---------------------------------
testcase 2
path cost: 21
14 -> 9 -> 4 -> 15 -> 12 -> 14 -> 1 -> 14 -> 6 -> 12 -> 3 -> 8 -> 6 -> 8 -> 6 -> 4 -> 10 -> 11 -> 10 -> 4 -> 13
nodes expanded: 583
duration: 0.2633938789367676
---------------------------------
testcase 3
path cost: 32
15 -> 14 -> 4 -> 20 -> 13 -> 10 -> 18 -> 10 -> 8 -> 9 -> 15 -> 5 -> 16 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 13 -> 19 -> 17 -> 8 -> 20 -> 2 -> 20 -> 4 -> 14 -> 16
nodes expanded: 14259
duration: 19.553257703781128




In [15]:
print("Testing ASTAR algorithm with alpha=1.2")
print()
run(partial(ASTAR, alpha=1.2))
print()
print("================================================================================")
print()


Testing ASTAR algorithm with alpha=1.2

testcase 1
path cost: 11
6 -> 4 -> 10 -> 9 -> 1 -> 3 -> 8 -> 9 -> 7 -> 9 -> 2
nodes expanded: 41
duration: 0.017982959747314453
---------------------------------
testcase 2
path cost: 21
14 -> 9 -> 4 -> 15 -> 12 -> 14 -> 1 -> 14 -> 6 -> 12 -> 3 -> 8 -> 6 -> 8 -> 6 -> 4 -> 10 -> 11 -> 10 -> 4 -> 13
nodes expanded: 534
duration: 0.24036836624145508
---------------------------------
testcase 3
path cost: 32
15 -> 14 -> 4 -> 20 -> 13 -> 10 -> 18 -> 10 -> 8 -> 9 -> 15 -> 5 -> 16 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 13 -> 19 -> 17 -> 8 -> 20 -> 2 -> 20 -> 4 -> 14 -> 16
nodes expanded: 11908
duration: 13.431561470031738




In [16]:
print("Testing ASTAR algorithm with alpha=4")
print()
run(partial(ASTAR, alpha=4))
print()
print("================================================================================")
print()


Testing ASTAR algorithm with alpha=4

testcase 1
path cost: 12
6 -> 9 -> 1 -> 4 -> 10 -> 9 -> 7 -> 9 -> 2 -> 8 -> 3 -> 8
nodes expanded: 16
duration: 0.00997304916381836
---------------------------------
testcase 2
path cost: 22
14 -> 9 -> 4 -> 15 -> 12 -> 3 -> 8 -> 6 -> 8 -> 6 -> 4 -> 7 -> 14 -> 1 -> 14 -> 12 -> 3 -> 5 -> 11 -> 8 -> 3 -> 13
nodes expanded: 63
duration: 0.030948400497436523
---------------------------------
testcase 3
path cost: 35
15 -> 14 -> 4 -> 20 -> 8 -> 20 -> 2 -> 20 -> 13 -> 10 -> 18 -> 10 -> 8 -> 9 -> 15 -> 5 -> 16 -> 12 -> 3 -> 19 -> 11 -> 19 -> 13 -> 6 -> 13 -> 1 -> 4 -> 14 -> 16 -> 14 -> 4 -> 1 -> 13 -> 19 -> 17
nodes expanded: 127
duration: 0.09068989753723145




In [17]:
print("Testing IDS algorithm:")
print()
run(IDS)
print()
print("================================================================================")
print()


Testing IDS algorithm:

::: limit= 0 , ::: limit= 1 , ::: limit= 2 , ::: limit= 3 , ::: limit= 4 , ::: limit= 5 , ::: limit= 6 , ::: limit= 7 , ::: limit= 8 , ::: limit= 9 , ::: limit= 10 , :::
testcase 1
path cost: 11
6 -> 4 -> 10 -> 9 -> 7 -> 9 -> 2 -> 9 -> 1 -> 3 -> 8
nodes expanded: 370
duration: 0.4597194194793701
---------------------------------
::: limit= 0 , ::: limit= 1 , ::: limit= 2 , ::: limit= 3 , ::: limit= 4 , ::: limit= 5 , ::: limit= 6 , ::: limit= 7 , ::: limit= 8 , ::: limit= 9 , ::: limit= 10 , ::: limit= 11 , ::: limit= 12 , ::: limit= 13 , ::: limit= 14 , ::: limit= 15 , ::: limit= 16 , ::: limit= 17 , ::: limit= 18 , ::: limit= 19 , ::: limit= 20 , :::
testcase 2
path cost: 21
14 -> 9 -> 4 -> 15 -> 4 -> 7 -> 14 -> 1 -> 14 -> 12 -> 3 -> 8 -> 6 -> 8 -> 6 -> 4 -> 10 -> 11 -> 5 -> 3 -> 13
nodes expanded: 3310
duration: 6.725808143615723
---------------------------------
::: limit= 0 , ::: limit= 1 , ::: limit= 2 , ::: limit= 3 , ::: limit= 4 , ::: limit= 5 , ::: lim

### 1.txt

| Algorithm | Path Lenght | Expanded States | Run Time |
| :---------------: | :-: | :-------------: | :------: |
| BFS               | 11  | 229             | 0.07977  |
| IDS               | 11  | 370             | 0.45971  |
| A*                | 11  | 104             | 0.04292  |
| A* (alpha=1.2)    | 11  | 41              | 0.01798  |
| A* (alpha=4)      | 12  | 16              | 0.00997  |

### 2.txt

| Algorithm | Path Lenght | Expanded States | Run Time |
| :---------------: | :-: | :-------------: | :------: |
| BFS               | 21  | 933             | 0.42193  |
| IDS               | 21  | 3310            | 6.72580  |
| A*                | 21  | 583             | 0.26339  |
| A* (alpha=1.2)    | 21  | 534             | 0.24036  |
| A* (alpha=4)      | 22  | 63              | 0.03094  |

### 3.txt

| Algorithm | Path Lenght | Expanded States | Run Time |
| :---------------: | :-: | :-------------: | :------: |
| BFS               | 32  | 21556           | 16.3255  |
| IDS               | 32  | 171482          | 817.043  |
| A*                | 32  | 14259           | 19.5532  |
| A* (alpha=1.2)    | 32  | 11908           | 13.4315  |
| A* (alpha=4)      | 35  | 127             | 0.09068  |

BFS and IDS will always find the optimal solution.
A* will find the optimal solution if the heuristic function is admissible and consistent.
Weighted A* may not always find the optimal solution, but it will find a close solution to the optimal solution based on the value of alpha and it will be much faster than A*.

The IDS is the slowest algorithm, but it uses the least amount of memory.